In [1]:
# Keras==1.0.6
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import  Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import tensorflow_hub as hub
from keras.layers import Dense,TimeDistributed
import keras.layers as layers
from keras.models import Model

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/user/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
raw = open('wikigold.conll.txt', 'r').readlines()
 
all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(' ')
    point.append(stripped_line)
    if line == '\n':
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]
 
lengths = [len(x) for x in all_x]
print('Input sequence length range: ', max(lengths), min(lengths))
 
short_x = [x for x in all_x if len(x) < 64]
 
X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]
 
all_text = [c for x in X for c in x]


Input sequence length range:  144 1


In [3]:
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}
print('Vocabulary size:', len(word2ind), len(label2ind))

Vocabulary size: 8285 5


In [4]:
maxlen = max([len(x) for x in X])
print('Maximum sequence length:', maxlen)
print(label2ind)

Maximum sequence length: 63
{'I-PER': 1, 'O': 2, 'I-MISC': 3, 'I-LOC': 4, 'I-ORG': 5}


In [5]:
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result
 
X_enc = [[word2ind[c] for c in x] for x in X]
max_label = max(label2ind.values()) + 1
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]
 
X_enc = pad_sequences(X_enc, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=11*32, train_size=45*32, random_state=42)
print('Training and testing tensor shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 
max_features = len(word2ind)
embedding_size = 300
hidden_size = 32
out_size = len(label2ind) + 1

Training and testing tensor shapes: (1440, 63) (352, 63) (1440, 63, 6) (352, 63, 6)


In [94]:
#elmo_model = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


In [95]:
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["word_emb"]

In [96]:
import tensorflow as tf
from keras import backend as K
sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [10]:
out_size

6

In [11]:
embedding.shape

NameError: name 'embedding' is not defined

In [33]:
dir(embedding)

#layer.get_output_at(0).get_shape().as_list()

['OVERLOADABLE_OPERATORS',
 '__abs__',
 '__add__',
 '__and__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__iter__',
 '__le__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_as_node_def_input',
 '_as_tf_output',
 '_c_api_shape',
 '_consumers',
 '_dtype',
 '_get_input_ops_without_shapes',
 '_id',
 '_keras_history',
 '_

In [98]:
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["word_emb"]

In [99]:
input_text = layers.Input(shape=(InputSize,300,), dtype=tf.string)

ElmoEmbedding(input_text)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<tf.Tensor 'module_1_apply_default_1/bilm/Reshape_1:0' shape=(?, ?, 512) dtype=float32>

In [91]:
tf.cast(main_input, tf.string)

<tf.Tensor 'main_input_10:0' shape=(?, 63) dtype=string>

In [93]:
tf.squeeze(tf.cast(main_input, tf.string))

<tf.Tensor 'Squeeze_5:0' shape=<unknown> dtype=string>

In [90]:
maxlen

63

In [87]:
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
    #return elmo_model(tf.cast(x, tf.string), signature="default", as_dict=True)["default"]

In [89]:
ElmoEmbedding(main_input)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<tf.Tensor 'module_apply_default_4/pooling/truediv:0' shape=(?, 1024) dtype=float32>

In [100]:
from keras.layers import Input, Embedding, LSTM, Dense, Lambda

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(maxlen,), dtype=tf.string, name='main_input')

#input_text = layers.Input(shape=(InputSize,300,), dtype=tf.string)

#ElmoEmbedding(input_text)

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
#x = Embedding(max_features,embedding_size,input_length=maxlen)(main_input)
x = Lambda(ElmoEmbedding, output_shape=(embedding_size,),name='Embedding_1')(main_input)


# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(hidden_size, return_sequences=True)(x)




INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [84]:
main_input.get_shape()

TensorShape([Dimension(None), Dimension(63)])

In [81]:
x.get_shape()

TensorShape([Dimension(None), Dimension(63), Dimension(300)])

In [77]:
hidden_size

32

In [83]:
maxlen

63

In [51]:
def ElmoEmbedding(x):
    #return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
    return elmo_model(tf.cast(x, tf.string), signature="default", as_dict=True)["default"]

In [52]:
embedding_size

300

In [53]:
input_text.get_shape()

TensorShape([Dimension(None), Dimension(63), Dimension(20), Dimension(22), Dimension(23)])

In [56]:
embedding.get_shape()

TensorShape([Dimension(None), Dimension(1024)])

In [55]:
InputSize = maxlen
batch_size = 32
input_text = layers.Input(shape=(InputSize,), dtype=tf.string)
#input_text = layers.Input(shape=(1,), dtype=tf.string)
#wordInputs = Input(shape=(MAX_SENTS+1, MAX_WORDS), name="wordInputs", dtype='float32')
#wordInp = Flatten()(wordInputs)
#wordEmbedding = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix], mask_zero=False, trainable=True, name='wordEmbedding')(wordInp) 

# l1 = Input(shape = (1000,), dtype='int32', name = 'l1')
# e1 = Embedding(output_dim=60,input_dim=1000,input_length=1000) (l1)

#input_text = layers.Input(shape=(InputSize,max_features), dtype=tf.string)

#embedding = layers.Lambda(ElmoEmbedding, output_shape=(embedding_size,),name='Embedding_1')(input_text)
embedding = layers.Lambda(ElmoEmbedding, output_shape=(embedding_size,),name='Embedding_1')(input_text)
#None, 63, 300



lstm_ = LSTM(hidden_size,batch_size=batch_size, return_sequences=True)(embedding)
#model.add(Embedding(max_features,embedding_size,input_length=maxlen,weights=[embedding_weights]))
timed_ = layers.TimeDistributed(layers.Dense(out_size))(lstm_)
pred = layers.Activation('softmax')(timed_)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

TypeError: Can't convert 'text': Shape TensorShape([Dimension(None), Dimension(63)]) is incompatible with TensorShape([Dimension(None)])

In [11]:
maxlen
embedding_size

300

In [16]:
embedding.get_shape()

TensorShape([Dimension(None), Dimension(1024)])

In [15]:
InputSize = maxlen
#model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
input_text = layers.Input(shape=(InputSize,max_features), dtype=tf.string)
embedding = layers.Lambda(ElmoEmbedding, output_shape=(embedding_size,))(input_text)
#dense = layers.TimeDistributed(layers.Dense(out_size))(embedding)
#pred = layers.Activation('softmax')(dense)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [20]:


input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(ElmoEmbedding, output_shape=(1024,))(input_text)


#model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
#model.add(Embedding(max_features,embedding_size,input_length=maxlen,weights=[embedding_weights]))

#model.add(TimeDistributed(Dense(out_size)))
dense = layers.TimeDistributed(Dense(out_size))

#dense = layers.Dense(256, activation='relu')(embedding)
#pred = layers.Dense(1, activation='sigmoid')(dense)
##pred = layers.Activation('softmax')(dense)

##model = Model(inputs=[input_text], outputs=pred)

##model.compile(loss='categorical_crossentropy', optimizer='adam')
##model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model.add(Embedding(max_features,embedding_size,input_length=maxlen,weights=[embedding_weights]))
model.add(LSTM(hidden_size, return_sequences=True))  


In [12]:
#model.add(TimeDistributedDense(out_size))
model.add(TimeDistributed(Dense(out_size)))

model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 63, 300)           2485500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 63, 32)            42624     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 63, 6)             198       
_________________________________________________________________
activation_1 (Activation)    (None, 63, 6)             0         
Total params: 2,528,322
Trainable params: 2,528,322
Non-trainable params: 0
_________________________________________________________________


In [13]:
batch_size = 32
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Raw test score:', score)

/home/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1440 samples, validate on 352 samples
Epoch 1/10
1440/1440 [==============================] - 4s 3ms/step - loss: 0.6218 - val_loss: 0.2713
Epoch 2/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.2324 - val_loss: 0.2049
Epoch 3/10
1440/1440 [==============================] - 4s 2ms/step - loss: 0.1810 - val_loss: 0.1683
Epoch 4/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.1438 - val_loss: 0.1407
Epoch 5/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.1134 - val_loss: 0.1195
Epoch 6/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0869 - val_loss: 0.1022
Epoch 7/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0650 - val_loss: 0.0901
Epoch 8/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0476 - val_loss: 0.0816
Epoch 9/10
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0348 - val_loss: 0.0773
Epoch 10/10
352/352 [==============

In [14]:
def score(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr
 
pr = model.predict_classes(X_train)
yh = y_train.argmax(2)
fyh, fpr = score(yh, pr)
print('Training accuracy:', accuracy_score(fyh, fpr))
print('Training confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)
 
pr = model.predict_classes(X_test)
yh = y_test.argmax(2)
fyh, fpr = score(yh, pr)
print('Testing accuracy:', accuracy_score(fyh, fpr))
print('Testing confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

Training accuracy: 0.9889061235500405
Training confusion matrix:
[[    0     0     0     0     0     0]
 [    1  1043     1    32     5    34]
 [    0     0  1205    16     1     4]
 [    0     7     1 24701    23    20]
 [    2     9     7    67   920    15]
 [    0    15     4    52    13  1458]]
Testing accuracy: 0.9320243737305348
Testing confusion matrix:
[[   0    0    0    0    0    0]
 [   1  164    5   55   11   32]
 [   0    1  214   49    3   16]
 [   0    1    5 6162   17   19]
 [   0   10    2  119  115   27]
 [   0   10   15   91   13  228]]


/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(array([0.        , 0.88172043, 0.8879668 , 0.95151328, 0.72327044,
        0.70807453]),
 array([0.        , 0.6119403 , 0.75618375, 0.99323017, 0.42124542,
        0.63865546]),
 array([0.        , 0.72246696, 0.81679389, 0.97192429, 0.53240741,
        0.67157585]),
 array([   0,  268,  283, 6204,  273,  357]))

In [15]:
from keras import backend as K
string = "John was a member of US Army"
wordlist = string.split(' ')
ip = []
for x in wordlist:
    ip.append(word2ind[x])
i=maxlen-len(ip)
temp=[0]*i
ip=temp+ip
input_layer = model.layers[0].input
output_layer = model.layers[3].output
op = K.function([input_layer], [output_layer])
out = op([[ip]])
temp = []
while i<maxlen:
    for j in label2ind:
        #print(out[0][i].tolist())
        if label2ind[j]==out[0][0][i].tolist().index(max(out[0][0][i])):
            temp.append(j)
    i=i+1
print(wordlist)
print(temp)

['John', 'was', 'a', 'member', 'of', 'US', 'Army']
['I-PER', 'O', 'O', 'O', 'O', 'I-ORG', 'I-ORG']
